In [2]:
import pandas as pd

df = pd.read_csv('assets/data/datos.csv', delimiter=';')

#print(df.head())

print(df.columns)

Index(['Categoria', 'Enlace', 'Nombre', 'ImagenURL', 'Ingredientes',
       'Calorias', 'Grasa Total(gr)', 'Colesterol(mg)', 'Sodio(mg)',
       'Carbohidrato Total(gr)', 'Proteina(gr)', 'Vitamina A(mcg)',
       'Vitamina C(mg)', 'Vitamina D(mcg)', 'Calcio(mg)', 'Hierro(mg)',
       'Potasio(mg)'],
      dtype='object')


In [10]:
import os
import pandas as pd
from langchain_openai import AzureOpenAIEmbeddings
from dotenv import load_dotenv
from sklearn.preprocessing import LabelEncoder

# Cargar variables de entorno
load_dotenv()

# Inicializar Azure OpenAI Embeddings
embeddings_model = AzureOpenAIEmbeddings(
    api_key=os.getenv("openai_embeddings_api_key"),
    base_url=os.getenv("openai_api_base_emb"),
    openai_api_type=os.getenv("openai_api_type"),
    #api_version=os.getenv("api_version_openai_emb"),
    #deployment_name= os.getenv("deploy_name_emb")
)

# Cargar el dataset
df = pd.read_csv('assets/data/datos.csv', delimiter=';')  # Asegúrate de reemplazar con tu ruta

# Columnas de texto que queremos convertir
columnas_texto = ['Categoria', 'Nombre', 'Ingredientes']

# Aplicar embeddings a las columnas de texto y convertir a números
for col in columnas_texto:
    df[col + "_emb"] = df[col].astype(str).apply(lambda x: sum(embeddings_model.embed_query(x)))

# Codificar las categorías con números (Label Encoding)
label_encoder = LabelEncoder()
df["Categoria_Num"] = label_encoder.fit_transform(df["Categoria"].astype(str))

# Columnas numéricas originales + nuevas columnas transformadas
columnas_numericas = [
    'Calorias', 'Grasa Total(gr)', 'Colesterol(mg)', 'Sodio(mg)',
    'Carbohidrato Total(gr)', 'Proteina(gr)', 'Vitamina A(mcg)',
    'Vitamina C(mg)', 'Vitamina D(mcg)', 'Calcio(mg)', 'Hierro(mg)',
    'Potasio(mg)', 'Categoria_Num', 'Nombre_emb', 'Ingredientes_emb'
]

# Reemplazar NaN con 0
df[columnas_numericas] = df[columnas_numericas].fillna(0)

# Crear la nueva columna con la suma total
df["Suma_Total"] = df[columnas_numericas].sum(axis=1)

# Guardar el dataset con la nueva columna
df.to_csv("dataset_numerico.csv", index=False)

print("Dataset convertido y guardado en dataset_numerico.csv ✅")


Dataset convertido y guardado en dataset_numerico.csv ✅


In [1]:
from langchain_openai import AzureOpenAIEmbeddings
import os
from dotenv import load_dotenv
import numpy as np
import pandas as pd

load_dotenv()

def similitud_cosenos(vect1: list, vect2: list) -> float:
    dot_product = np.dot(vect1, vect2)
    norm_A = np.linalg.norm(vect1)
    norm_B = np.linalg.norm(vect2)
    cosine_similarity = dot_product / (norm_A * norm_B)
    return cosine_similarity


df = pd.read_csv('assets/data/datos.csv', delimiter=";") 

#respuestas = df["Ingredientes"].dropna().tolist()

respuestas = df[["Ingredientes", "Nombre", "Categoria"]].dropna().values.tolist()

embeddings = AzureOpenAIEmbeddings(
    api_key=os.getenv("openai_embeddings_api_key"),
    base_url=os.getenv("openai_api_base_emb"),
    openai_api_type=os.getenv("openai_api_type"),
)

query = [embeddings.embed_query(text[1]) for text in respuestas]


In [20]:

query_client = "Receta de torta de fresa"
query_embeddings = embeddings.embed_query(query_client)


response_option = [similitud_cosenos(query_embeddings, q) for q in query]

pos = np.argmax(response_option)
request = respuestas[pos]

print(f"{query_client} - rta: {request}")

Receta de torta de fresa - rta: [' | 1 1/2 tazas de fresas picadas finamente (alrededor de 1/2 libra) | 1/2 chile jalapeño, picado finamente | 1/4 taza de cebolla finamente picada | 2 cucharadas de cilantro picado | 1 1/2 cucharaditas de jugo de limón', 'Salsa de fresas', 'Postres']


In [4]:

query_client = "Receta de spagheti"
query_embeddings = embeddings.embed_query(query_client)


response_option = [similitud_cosenos(query_embeddings, q) for q in query]


pos = np.argmax(response_option)
request_ingredientes, request_nombre, request_categoria = respuestas[pos]

print(f"{query_client}\nReceta: {request_nombre}\nCategoría: {request_categoria}\nIngredientes: {request_ingredientes}")

Receta de spagheti
Receta: Calabaza espagueti
Categoría: Cena
Ingredientes:  | 1 calabaza tipo espagueti entera | Condimentos de su elección para cubrir la calabaza
